# Task 3: PostgreSQL Database Integration

## Overview
This notebook documents the PostgreSQL database integration for the Bank Reviews Analysis project. We will:
1. Connect to the PostgreSQL database
2. Load processed review data into the database
3. Verify data integrity and foreign key constraints
4. Run analytical queries to validate the data

## Prerequisites
- PostgreSQL database named `bank_reviews` created
- Tables `banks` and `reviews` with proper schema
- Environment variables configured in `.env` file
- Processed CSV data available in `data/processed/`

In [1]:
# Database Integration Analysis

import pandas as pd
import sys
import os

# Add project root to path
sys.path.append('..')

# Import our database manager
from src.database.db_manager import DatabaseManager

print("Database integration notebook loaded successfully.")

Database integration notebook loaded successfully.


## 1. Database Connection Test

First, we test the connection to the PostgreSQL database using our custom `DatabaseManager` class. This verifies that:
- Environment variables are properly configured
- PostgreSQL service is running
- User has proper permissions to access the database

In [2]:
print("Testing Database Connection...")

db = DatabaseManager()

if db.connect():
    # Get PostgreSQL version
    db.cursor.execute("SELECT version();")
    version = db.cursor.fetchone()
    print(f"PostgreSQL Version: {version[0]}")
    
    # Count tables
    db.cursor.execute("""
        SELECT COUNT(*) 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    table_count = db.cursor.fetchone()[0]
    print(f"Tables in database: {table_count}")
    
    db.disconnect()
    print("Database connection test: PASSED")
else:
    print("Database connection test: FAILED")

Testing Database Connection...
 Connected to PostgreSQL database successfully!
PostgreSQL Version: PostgreSQL 16.10 (Ubuntu 16.10-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0, 64-bit
Tables in database: 2
 Database connection closed.
Database connection test: PASSED


## 2. Data Loading Process

Now we load the processed review data from the CSV file into the PostgreSQL database. The process includes:
1. Reading the `sentiment_themes_analysis.csv` file
2. Inserting bank information into the `banks` table
3. Inserting review data with sentiment analysis into the `reviews` table
4. Maintaining referential integrity through foreign key constraints

In [4]:
print("Loading data into database...")

db = DatabaseManager()

if db.connect():
    # Load data from CSV into database
    success = db.load_data_from_csv("../data/processed/sentiment_themes_analysis.csv")
    
    if success:
        print("Data loaded successfully into database.")
    else:
        print("Failed to load data into database.")
    
    db.disconnect()
else:
    print("Cannot connect to database.")

Loading data into database...
 Connected to PostgreSQL database successfully!
 Loading data from CSV...
 Loaded 1346 reviews from CSV
 Inserted 3 banks
 Inserted 1346 reviews
Data loaded successfully into database.
 Database connection closed.


## 3. Data Insertion Verification

After loading the data, we verify that the insertion was successful and meets the project requirements:
- Minimum requirement: At least 400 reviews per bank (1200 total)
- KPI requirement: Over 1000 review entries in the database
- Check that all three banks are properly represented

In [5]:
print("Verifying data insertion...")

db = DatabaseManager()

if db.connect():
    # Get counts from database
    db.cursor.execute("SELECT COUNT(*) FROM banks;")
    bank_count = db.cursor.fetchone()[0]
    
    db.cursor.execute("SELECT COUNT(*) FROM reviews;")
    review_count = db.cursor.fetchone()[0]
    
    print(f"Banks in database: {bank_count}")
    print(f"Reviews in database: {review_count}")
    
    # Check if we meet the requirement (>1000 reviews)
    if review_count >= 1000:
        print(f"SUCCESS: Database contains {review_count} reviews (exceeds 1000 requirement)")
    else:
        print(f"WARNING: Database contains only {review_count} reviews")
    
    db.disconnect()
else:
    print("Cannot connect to database.")

Verifying data insertion...
 Connected to PostgreSQL database successfully!
Banks in database: 3
Reviews in database: 1346
SUCCESS: Database contains 1346 reviews (exceeds 1000 requirement)
 Database connection closed.


## 4. Analytical Queries

To demonstrate the database is working correctly, we run several SQL queries that provide business insights:

1. **Reviews per bank**: Distribution of reviews across different banks
2. **Average rating per bank**: Performance comparison between banks
3. **Sentiment distribution**: Overall sentiment analysis results

These queries validate that our data is properly structured and can support business analysis.

In [6]:
print("Running sample SQL queries...")

db = DatabaseManager()

if db.connect():
    print("\n1. Reviews per bank:")
    db.cursor.execute("""
        SELECT b.bank_name, COUNT(r.review_id) as review_count
        FROM banks b
        LEFT JOIN reviews r ON b.bank_id = r.bank_id
        GROUP BY b.bank_name
        ORDER BY review_count DESC;
    """)
    
    for bank, count in db.cursor.fetchall():
        print(f"   {bank}: {count} reviews")
    
    print("\n2. Average rating per bank:")
    db.cursor.execute("""
        SELECT b.bank_name, 
               ROUND(AVG(r.rating), 2) as avg_rating,
               COUNT(r.review_id) as review_count
        FROM banks b
        JOIN reviews r ON b.bank_id = r.bank_id
        GROUP BY b.bank_name
        ORDER BY avg_rating DESC;
    """)
    
    for bank, avg_rating, count in db.cursor.fetchall():
        print(f"   {bank}: {avg_rating} stars ({count} reviews)")
    
    print("\n3. Sentiment distribution:")
    db.cursor.execute("""
        SELECT sentiment_label, COUNT(*) as count,
               ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM reviews), 1) as percentage
        FROM reviews
        WHERE sentiment_label IS NOT NULL
        GROUP BY sentiment_label
        ORDER BY count DESC;
    """)
    
    for sentiment, count, pct in db.cursor.fetchall():
        print(f"   {sentiment}: {count} reviews ({pct}%)")
    
    db.disconnect()
else:
    print("Cannot connect to database.")

Running sample SQL queries...
 Connected to PostgreSQL database successfully!

1. Reviews per bank:
   Bank of Abyssinia: 450 reviews
   Commercial Bank of Ethiopia: 450 reviews
   Dashen Bank: 446 reviews

2. Average rating per bank:
   Dashen Bank: 3.98 stars (446 reviews)
   Commercial Bank of Ethiopia: 2.67 stars (450 reviews)
   Bank of Abyssinia: 2.07 stars (450 reviews)

3. Sentiment distribution:
   NEGATIVE: 907 reviews (67.4%)
   POSITIVE: 439 reviews (32.6%)
 Database connection closed.


## 5. Data Quality Assurance

Data quality is critical for reliable analysis. We perform several checks:

1. **Missing data check**: Identify reviews with NULL values in critical fields
2. **Data validation**: Ensure ratings are within valid range (1-5 stars)
3. **Referential integrity**: Verify foreign key constraints are maintained (no orphaned reviews)

These checks ensure our database contains clean, reliable data for decision-making.

In [7]:
print("Running data quality checks...")

db = DatabaseManager()

if db.connect():
    print("\n1. Missing data check:")
    
    checks = [
        ("Reviews with NULL sentiment", "SELECT COUNT(*) FROM reviews WHERE sentiment_label IS NULL"),
        ("Reviews with NULL rating", "SELECT COUNT(*) FROM reviews WHERE rating IS NULL"),
        ("Reviews with NULL date", "SELECT COUNT(*) FROM reviews WHERE review_date IS NULL"),
        ("Reviews outside rating range", "SELECT COUNT(*) FROM reviews WHERE rating < 1 OR rating > 5")
    ]
    
    for check_name, query in checks:
        db.cursor.execute(query)
        count = db.cursor.fetchone()[0]
        status = "PASS" if count == 0 else "FAIL"
        print(f"   {check_name}: {count} ({status})")
    
    print("\n2. Foreign key integrity check:")
    db.cursor.execute("""
        SELECT COUNT(*) 
        FROM reviews r
        LEFT JOIN banks b ON r.bank_id = b.bank_id
        WHERE b.bank_id IS NULL;
    """)
    
    orphaned_reviews = db.cursor.fetchone()[0]
    if orphaned_reviews == 0:
        print("   Foreign key integrity: PASS (no orphaned reviews)")
    else:
        print(f"   Foreign key integrity: FAIL ({orphaned_reviews} orphaned reviews)")
    
    db.disconnect()
else:
    print("Cannot connect to database.")

Running data quality checks...
 Connected to PostgreSQL database successfully!

1. Missing data check:
   Reviews with NULL sentiment: 0 (PASS)
   Reviews with NULL rating: 0 (PASS)
   Reviews with NULL date: 0 (PASS)
   Reviews outside rating range: 0 (PASS)

2. Foreign key integrity check:
   Foreign key integrity: PASS (no orphaned reviews)
 Database connection closed.
